In [1]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp
from sklearn.preprocessing import normalize

# load train, test data

In [2]:
train = pd.read_csv("./image/dtw_sample_train_skeleton.csv", header=[0,1])

In [3]:
test = pd.read_csv("./image/dtw_sample_test_skeleton.csv", header=[0,1])

- 원본 비율인 (640 * 720) 로 통일시키는 작업이 필요

# extract x, y from skeleton & change to (640 * 720)

In [4]:
def origin_rate(df):
    df_xy = df[[(a, b) for a, b in train.columns if b in ('x', 'y')]]
    df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
    df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)    
    return df_xy

# bounding box & perspective transform

In [5]:
def box_perspective(df_xy):
    for idx in range(len(df_xy)):
        x_min = df_xy.loc[idx, (slice(None), 'x')].min()
        x_max = df_xy.loc[idx, (slice(None), 'x')].max()
        y_min = df_xy.loc[idx, (slice(None), 'y')].min()
        y_max = df_xy.loc[idx, (slice(None), 'y')].max()

        x_rate = 640/((x_max+20)-(x_min-20))
        y_rate = 720/((y_max+20)-(y_min-20))

        df_xy.loc[idx, (slice(None), 'x')] = df_xy.loc[idx, (slice(None), 'x')].apply(lambda x : (x-(x_min-20))*x_rate)
        df_xy.loc[idx, (slice(None), 'y')] = df_xy.loc[idx, (slice(None), 'y')].apply(lambda x : (x-(y_min-20))*y_rate)
        
    return df_xy

# cosine similarity

In [6]:
def cosine_similarity(train, test):
    train = normalize(train)
    test = normalize(test)
    cossim_list = []
    for x, y in zip(train, test):
        cossim = np.dot(x, y) / (np.linalg.norm(x)*np.linalg.norm(y))
        cossim_list.append(cossim)
    return cossim_list

# cosine similarity by using preprocessed data

In [7]:
def cossim_score(train, test):
    train = origin_rate(train)
    test = origin_rate(test)
    
    train = box_perspective(train)
    test = box_perspective(test)
    
    cossim_list = cosine_similarity(train, test)
    print(cossim_list)
    return np.mean(cossim_list)

In [8]:
cossim_score(train,test)

C:\Users\EOM\AppData\Local\Temp\ipykernel_23808\56938711.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
C:\Users\EOM\AppData\Local\Temp\ipykernel_23808\56938711.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
C:\Users\EOM\AppData\Local\Temp\ipykernel_23808\56938711.py:3: SettingWithCopyWarning: 
A value is trying to b

[0.9586796055060256, 0.9626725331075946, 0.9677971541480507, 0.9736866657523419, 0.9786281832814725, 0.979442644202207, 0.9797960213857231, 0.9824086891796411, 0.9886410098974926, 0.99285881033121, 0.9966688053493691, 0.9981575826847819, 0.9969489507212883, 0.9954892771794178, 0.9947657701778377, 0.9951085791307888, 0.9951653713764911, 0.9940564377391198, 0.9918176745825792, 0.9915616778637463, 0.9911800136384834, 0.99198201930028, 0.993051144668597, 0.9939795314946509, 0.9949727353755022, 0.9956320295452633, 0.9944086832242485, 0.9927804757971237, 0.9917302793905592, 0.989276282536754]


0.9881114879522879

# euclidean distance of cosine similarity

In [15]:
def euclid_distance(train, test):
    train = normalize(train)
    test = normalize(test)
    euclid_list = []
    for x, y in zip(train, test):
        cossim = np.dot(x, y) / (np.linalg.norm(x)*np.linalg.norm(y))
        euc_d = np.sqrt(2*(1-cossim))
        score = 100 - (100 * (euc_d)/2)
        euclid_list.append(score)
    return euclid_list

# euclidean distance of cosine similarity by using preprocessed data

In [16]:
def cossim_euclid(train, test):
    train = origin_rate(train)
    test = origin_rate(test)
    
    train = box_perspective(train)
    test = box_perspective(test)
    
    euclid_list = euclid_distance(train, test)
    print(euclid_list)
    
    return np.mean(euclid_list)

In [17]:
cossim_euclid(train, test)

C:\Users\EOM\AppData\Local\Temp\ipykernel_23808\56938711.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
C:\Users\EOM\AppData\Local\Temp\ipykernel_23808\56938711.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
C:\Users\EOM\AppData\Local\Temp\ipykernel_23808\56938711.py:3: SettingWithCopyWarning: 
A value is trying to b

[85.62634449870626, 86.3384724696677, 87.31086176055496, 88.52974842306017, 89.66273326296368, 89.86161852222135, 89.9491347103155, 90.62148444039279, 92.46375753358897, 94.02455454849182, 95.91882697583715, 96.96485806327112, 96.09420348794788, 95.25093544970055, 94.88422546325472, 95.05458754540572, 95.08338092612976, 94.5485954741552, 93.60378025024906, 93.50449303893309, 93.35922204801403, 93.66834117323113, 94.10557240633028, 94.51343982747427, 94.98638622124831, 95.32668722706435, 94.71260140723648, 93.9918704229701, 93.56971205565382, 92.67752860598078]


92.87359860800167

# fastdtw

In [18]:
from fastdtw import fastdtw

In [21]:
def fast_dtw(train, test):
    train = origin_rate(train)
    test = origin_rate(test)
    
    train = box_perspective(train)
    test = box_perspective(test)
    
    train = normalize(train)
    test = normalize(test)
    
    results = [fastdtw(a, b)[0] for a, b in zip(train, test)]
    print(results)
    return np.mean(results)

# fastdtw by using preprocessed data

In [22]:
fast_dtw(train, test)

C:\Users\EOM\AppData\Local\Temp\ipykernel_23808\56938711.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
C:\Users\EOM\AppData\Local\Temp\ipykernel_23808\56938711.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
C:\Users\EOM\AppData\Local\Temp\ipykernel_23808\56938711.py:3: SettingWithCopyWarning: 
A value is trying to b

[1.654210453061402, 1.718550431415356, 1.6219457901384786, 1.3082799334389559, 1.189930369701576, 1.1241371972546954, 1.3934873906264635, 1.0568569972450632, 0.7628028682836626, 0.6790325082281086, 0.4167896548948705, 0.3631588204192593, 0.44095375825609284, 0.5788348343222662, 0.6056744979632591, 0.5120699812190866, 0.6101566887876624, 0.7096447820984977, 0.8639263789511256, 0.8751101658752976, 0.9261620916694813, 0.8687795739473091, 0.7827762239063389, 0.716825150762717, 0.6980163740535056, 0.6712917185236912, 0.6599409168706601, 0.7081055434605501, 0.7187835267665338, 0.8346138241862717]


0.8690282815442745

# use wrong data

In [25]:
wrong_data = pd.read_csv('./image/knee_clap2_skeleton.csv', header=[0,1])

In [27]:
cossim_score(train, wrong_data)

C:\Users\EOM\AppData\Local\Temp\ipykernel_23808\56938711.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
C:\Users\EOM\AppData\Local\Temp\ipykernel_23808\56938711.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
C:\Users\EOM\AppData\Local\Temp\ipykernel_23808\56938711.py:3: SettingWithCopyWarning: 
A value is trying to b

[0.978687016333144, 0.9762785371354709, 0.9761183324329543, 0.9725974306670826, 0.9681742947683392, 0.9625802591364567, 0.9649717738876231, 0.9704003599500306, 0.9793700521201956, 0.9826893848263936, 0.9833912447765966, 0.9847078027990428, 0.9831056547766419, 0.9713639347901498, 0.9726448461033165, 0.9818757319451702, 0.9842921917791181, 0.9879431562275357, 0.9892379967449529, 0.9927298981413675, 0.993456912449082, 0.9950211737393284, 0.9956648021235719, 0.996095489170175, 0.9960855465293456, 0.9968312582346662, 0.9921430717309432, 0.988648027959295, 0.9842132732464578, 0.9860987778112968]


0.9829139410778581

In [28]:
cossim_euclid(train, wrong_data)

C:\Users\EOM\AppData\Local\Temp\ipykernel_23808\56938711.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
C:\Users\EOM\AppData\Local\Temp\ipykernel_23808\56938711.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
C:\Users\EOM\AppData\Local\Temp\ipykernel_23808\56938711.py:3: SettingWithCopyWarning: 
A value is trying to b

[89.67697145531992, 89.10930147682686, 89.07258777956883, 88.29475131983148, 87.38538442288849, 86.32159715764607, 86.76591028586083, 87.83454890890408, 89.84373398344539, 90.69660944235747, 90.88716421101438, 91.25580272381815, 90.80914986974598, 88.03420182147255, 88.30488266482898, 90.48047583782945, 91.13777448355044, 92.23570873406197, 92.66446891661992, 93.97086164587654, 94.28025894339699, 95.01059810164004, 95.3442520061605, 95.58156655033426, 95.57594446765502, 96.01958433996286, 93.73225388634127, 92.4660859970713, 91.11553975934885, 91.66296749775343]


91.18569795637109

In [29]:
fast_dtw(train, wrong_data)

C:\Users\EOM\AppData\Local\Temp\ipykernel_23808\56938711.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'x')] = df_xy.loc[:, (slice(None), 'x')].apply(lambda x : x*640)
C:\Users\EOM\AppData\Local\Temp\ipykernel_23808\56938711.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy.loc[:, (slice(None), 'y')] = df_xy.loc[:, (slice(None), 'y')].apply(lambda x : x*720)
C:\Users\EOM\AppData\Local\Temp\ipykernel_23808\56938711.py:3: SettingWithCopyWarning: 
A value is trying to b

[1.3126851846435648, 1.4419160547607892, 1.3528518628793358, 1.4819204802276307, 1.2911100421409794, 1.3542528871045347, 1.3512724895828316, 1.4340117307447944, 1.2800335050450642, 1.1864177580032802, 1.2465876047183142, 1.2062763615932184, 1.204023405701856, 1.5572378519662715, 1.5486174880691945, 2.3894491940535523, 2.43947175025441, 1.0645790706154634, 1.0481202959786096, 0.7635206023524856, 0.701608381516172, 0.6201741424643674, 0.5722345927583897, 0.888081866170203, 0.543420137047499, 0.4939885644640652, 0.6392635715930504, 0.7693115347309056, 1.3738895642492355, 0.9604511582601798]


1.1838926377896746

# 결과

## 동일한 동작으로 정확도 확인
- cosine similarity : 0.9881114879522879
- euclidean distance of cosine similarity : 92.87359860800167
- fastdtw : 0.8690282815442745

## 다른 동작으로 정확도 확인
- cosine similarity : 0.9829139410778581
- euclidean distance of cosine similarity : 91.18569795637109
- fastdtw : 1.1838926377896746

- bounding box와 perspecive transform과 같은 preprocessing 을 했음에도 불구하고 성능이 크게 좋아지지 않았음